## <b>Otimização de Hiperparâmetros</b>
### Foi utilizado as abordagens de otimização:
#### 1. Grid Search
#### 2. Random Search
#### 3. Bayesian Optization
#### 4. Hyperopt
#### 5. Optuna


In [ ]:
# Instalação Bibliotecas 
!pip install bayesian-optimization
!pip install optuna

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=87793507ee2cedb0ce67ec18420e3dc239b969efc8fedd0f7124bea325718ccc
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization
     |████████████████████████████████| 235kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 18.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 133kB 36.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 112kB 31.4MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=d0b4e49190e6fd0996d5f14427fa76bb5c8dad91cc2e8c4cabbf513ee079b31a
  Stor

In [ ]:
#imports
from time import time
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from hyperopt import hp, STATUS_OK, Trials, fmin, tpe, space_eval
import optuna

### Obtenção dos dados e definição do pipeline

In [ ]:
#Categorias que vamos obter do dataset fetch_20newsgroups
categorias = [
        'alt.atheism',
        'talk.religion.misc',
        'comp.graphics'
]

data = fetch_20newsgroups(subset='train', categories=categorias)

print("%d Quantidade de frases" % len(data.filenames))
print("%d Classes" % len(data.target_names))

# #############################################################################
# Definição do pipeline
# #############################################################################

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel = 'rbf')),
])




1441 Quantidade de frases
3 Classes


### 1. GridSearch

In [ ]:


parameters = {
    'clf__C':[0.1, 10, 100, 1000],
    'clf__gamma':[0.001,0.005,0.01,1]
}

t0 = time()


grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv= 5)
grid_search.fit(data.data, data.target)


print("Pronto em  %0.3fs" % (time() - t0))


print("Melhor score: %0.3f" % grid_search.best_score_)
print("Melhores parametros encontrados: ", grid_search.best_params_)


Pronto em  183.241s
Melhor score: 0.965
Melhores parametros encontrados:  {'clf__C': 100, 'clf__gamma': 0.01}


### Random Search

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel = 'rbf')),
])

parameters = {
    'clf__C':[0.1, 1, 10, 100, 1000],
    'clf__gamma':[0.001,0.005,0.01,1,50]
}

t0 = time()


random_search = RandomizedSearchCV(pipeline, parameters,n_iter = 5,  n_jobs=-1, cv= 5)
random_search.fit(data.data, data.target)


print("Pronto em  %0.3fs" % (time() - t0))

print("Melhor score: %0.3f" % random_search.best_score_)
print("Melhores parametros encontrados: ", random_search.best_params_)


Pronto em  65.700s
Melhor score: 0.405
Melhores parametros encontrados:  {'clf__gamma': 0.001, 'clf__C': 1}


### Bayesian Optization

In [ ]:
parameters = {
    'clf__C':( 0.1,1000),
    'clf__gamma': (0.001,50)
}

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel = 'rbf')),
])

def objective(clf__C,clf__gamma): 
    pipeline.set_params(clf__C =clf__C, clf__gamma = clf__gamma )
    score = cross_val_score(pipeline, data.data, data.target, cv=5)
    accuracy = score.mean()
    
    return  accuracy

optimizer = BayesianOptimization(f=objective,pbounds=parameters)
t0 = time()
optimizer.maximize(init_points=10, n_iter=10)

print("Pronto em  %0.3fs" % (time() - t0)) 
print("Melhor score: %0.3f" % optimizer.max['target'])
print("Melhores parametros encontrados:")
print("C: %0.3f" % optimizer.max['params']['clf__C'])
print("Gamma: %0.3f" % optimizer.max['params']['clf__gamma'])



|   iter    |  target   |  clf__C   | clf__g... |
-------------------------------------------------
|  1        |  0.4053   |  149.4    |  33.15    |
|  2        |  0.4053   |  946.3    |  31.38    |
|  3        |  0.5829   |  786.9    |  4.278    |
|  4        |  0.4053   |  599.9    |  41.67    |
|  5        |  0.4053   |  912.1    |  25.71    |
|  6        |  0.4053   |  962.1    |  27.67    |
|  7        |  0.9605   |  686.3    |  0.2935   |
|  8        |  0.9563   |  444.3    |  0.7005   |
|  9        |  0.4053   |  914.4    |  23.87    |
|  10       |  0.4053   |  570.1    |  34.82    |
|  11       |  0.4053   |  0.1      |  0.001    |
|  12       |  0.9535   |  314.4    |  0.001    |
|  13       |  0.4053   |  360.9    |  49.81    |
|  14       |  0.9591   |  379.6    |  0.001    |
|  15       |  0.9639   |  630.0    |  0.01488  |
|  16       |  0.9507   |  242.6    |  0.001    |
|  17       |  0.9625   |  660.1    |  0.07841  |
|  18       |  0.9632   |  499.7    |  0.001    |


### Hyperopt

In [ ]:
parameters = {
    'clf__C':hp.uniform('clf__C', 0.1,1000),
    'clf__gamma':hp.uniform('gamma',  0.001,50)
}

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel = 'rbf')),
])

def objective(hyperparams):
    
    pipeline.set_params(clf__C =hyperparams['clf__C'], clf__gamma = hyperparams['clf__gamma'] )    
    score = cross_val_score(pipeline, data.data, data.target, cv=5)
    accuracy = score.mean()
    return  -accuracy
        
trials = Trials()
t0 = time()

best = fmin(objective, parameters, algo=tpe.suggest, max_evals=10, trials=trials)

print("Pronto em  %0.3fs" % (time() - t0))


print("Melhores parametros encontrados:")
print(best)



100%|██████████| 10/10 [02:59<00:00, 17.97s/it, best loss: -0.4129036908881199]
Pronto em  179.688s
Melhores parametros encontrados:
{'clf__C': 938.5872758357903, 'gamma': 11.44159737090584}


### Optuna

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel = 'rbf')),
])


def objective(trial):
    
    pipeline.set_params(clf__C =trial.suggest_loguniform("clf__C", 0.1, 1e3), 
                        clf__gamma = trial.suggest_loguniform("clf__gamma", 1e-4, 50))  
    score =cross_val_score(pipeline, data.data, data.target, cv=5)
    accuracy = score.mean()
    
    return accuracy

t0 = time()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, n_jobs=-1)

print("Pronto em  %0.3fs" % (time() - t0))

print("Melhor score: %0.3f" % study.best_trial.value)
print("Melhores parametros encontrados:")
print(study.best_trial.params)




[I 2020-08-12 21:04:36,993] Trial 1 finished with value: 0.5794622260668973 and parameters: {'clf__C': 3.488498438956914, 'clf__gamma': 0.006561502452887164}. Best is trial 1 with value: 0.5794622260668973.
[I 2020-08-12 21:04:36,998] Trial 0 finished with value: 0.6542027104959631 and parameters: {'clf__C': 7.9230303574407674, 'clf__gamma': 0.3352618216143795}. Best is trial 0 with value: 0.6542027104959631.
[I 2020-08-12 21:04:59,809] Trial 3 finished with value: 0.9611423490965013 and parameters: {'clf__C': 488.85874131071864, 'clf__gamma': 0.13556980049739173}. Best is trial 3 with value: 0.9611423490965013.
[I 2020-08-12 21:04:59,815] Trial 2 finished with value: 0.9611423490965013 and parameters: {'clf__C': 16.50657885396258, 'clf__gamma': 0.00010096328976993472}. Best is trial 3 with value: 0.9611423490965013.
[I 2020-08-12 21:05:21,933] Trial 5 finished with value: 0.963917723952326 and parameters: {'clf__C': 211.0962439533668, 'clf__gamma': 0.01808855218916397}. Best is trial 

Pronto em  113.164s
Melhor score: 0.964
Melhores parametros encontrados:
{'clf__C': 211.0962439533668, 'clf__gamma': 0.01808855218916397}
